<a href="https://colab.research.google.com/github/alamin19/Time_Series_Prediction_2020/blob/main/Prediction_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [122]:
df = pd.read_csv('cqmpredict.csv')
df.head()

,year,IVD,HbA1c <=9% of Total Diabetics,Hypertensive Patients w BP < 140/90,Children Weight,Depression,Dental Sealant,HbA1c >9% of Total Diabetics,Adult Weight,CAD,Tobacco,Colorectal Cancer,Pap Tests,Early Entry Prenatal,Low Birth Weight,Immunization,HIV and Received Follow-up Treatment,Asthmatic,Statin Therapy
0,2016,78.40%,67.90%,62.39%,62.92%,60.34%,48.71%,32.10%,62.46%,79.46%,85.19%,39.89%,54.36%,74.06%,7.80%,42.80%,83.17%,87.38%,NaN
1,2017,79.27%,67.05%,62.71%,65.85%,66.15%,50.71%,32.95%,63.85%,80.72%,87.50%,42.02%,55.67%,73.97%,8.03%,40.24%,84.52%,86.62%,NaN
2,2018,80.86%,67.21%,63.26%,69.16%,70.57%,52.80%,32.79%,70.15%,80.63%,88.09%,44.11%,55.95%,73.82%,8.00%,39.44%,85.55%,86.58%,NaN
3,2019,80.78%,68.05%,64.62%,71.21%,71.61%,56.80%,31.95%,72.43%,NaN,87.17%,45.56%,56.53%,73.81%,8.05%,39.75%,87.21%,85.95%,70.09%
4,2020,78.80%,35.60%,57.98%,65.13%,64.21%,48.68%,35.60%,65.72%,NaN,83.43%,40.09%,NaN,73.48%,8.18%,40.42%,81.41%,NaN,71.92%


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   year                                  5 non-null      int64 
 1   IVD                                   5 non-null      object
 2   HbA1c <=9% of Total Diabetics         5 non-null      object
 3   Hypertensive Patients w BP < 140/90   5 non-null      object
 4   Children Weight                       5 non-null      object
 5   Depression                            5 non-null      object
 6   Dental Sealant                        5 non-null      object
 7   HbA1c >9% of Total Diabetics          5 non-null      object
 8   Adult Weight                          5 non-null      object
 9   CAD                                   3 non-null      object
 10  Tobacco                               5 non-null      object
 11  Colorectal Cancer                   

In [124]:
df.replace('%', '', regex = True, inplace = True)


In [125]:
df.head()

,year,IVD,HbA1c <=9% of Total Diabetics,Hypertensive Patients w BP < 140/90,Children Weight,Depression,Dental Sealant,HbA1c >9% of Total Diabetics,Adult Weight,CAD,Tobacco,Colorectal Cancer,Pap Tests,Early Entry Prenatal,Low Birth Weight,Immunization,HIV and Received Follow-up Treatment,Asthmatic,Statin Therapy
0,2016,78.40,67.90,62.39,62.92,60.34,48.71,32.10,62.46,79.46,85.19,39.89,54.36,74.06,7.80,42.80,83.17,87.38,NaN
1,2017,79.27,67.05,62.71,65.85,66.15,50.71,32.95,63.85,80.72,87.50,42.02,55.67,73.97,8.03,40.24,84.52,86.62,NaN
2,2018,80.86,67.21,63.26,69.16,70.57,52.80,32.79,70.15,80.63,88.09,44.11,55.95,73.82,8.00,39.44,85.55,86.58,NaN
3,2019,80.78,68.05,64.62,71.21,71.61,56.80,31.95,72.43,NaN,87.17,45.56,56.53,73.81,8.05,39.75,87.21,85.95,70.09
4,2020,78.80,35.60,57.98,65.13,64.21,48.68,35.60,65.72,NaN,83.43,40.09,NaN,73.48,8.18,40.42,81.41,NaN,71.92


In [126]:
df = df.astype('float') #convert object to number

In [127]:
df.year = df.year.astype('int')

In [128]:
df.describe() #std needs to be same to employ regular machine learning. Scalling needs to be performed. Typical machine learning, no variance is expected.

,year,IVD,HbA1c <=9% of Total Diabetics,Hypertensive Patients w BP < 140/90,Children Weight,Depression,Dental Sealant,HbA1c >9% of Total Diabetics,Adult Weight,CAD,Tobacco,Colorectal Cancer,Pap Tests,Early Entry Prenatal,Low Birth Weight,Immunization,HIV and Received Follow-up Treatment,Asthmatic,Statin Therapy
count,5.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,5.000000,5.000000,4.000000,5.000000,5.000000,5.000000,5.000000,4.000000,2.000000
mean,2018.000000,79.622000,61.162000,62.19200,66.854000,66.576000,51.540000,33.078000,66.922000,80.270000,86.276000,42.334000,55.627500,73.828000,8.012000,40.530000,84.372000,86.632500,71.005000
std,1.581139,1.136495,14.296047,2.50421,3.307148,4.635739,3.396049,1.473828,4.229346,0.702922,1.927895,2.483371,0.917764,0.221066,0.137004,1.327177,2.218225,0.585228,1.294005
min,2016.000000,78.400000,35.600000,57.98000,62.920000,60.340000,48.680000,31.950000,62.460000,79.460000,83.430000,39.890000,54.360000,73.480000,7.800000,39.440000,81.410000,85.950000,70.090000
25%,2017.000000,78.800000,67.050000,62.39000,65.130000,64.210000,48.710000,32.100000,63.850000,80.045000,85.190000,40.090000,55.342500,73.810000,8.000000,39.750000,83.170000,86.422500,70.547500
50%,2018.000000,79.270000,67.210000,62.71000,65.850000,66.150000,50.710000,32.790000,65.720000,80.630000,87.170000,42.020000,55.810000,73.820000,8.030000,40.240000,84.520000,86.600000,71.005000
75%,2019.000000,80.780000,67.900000,63.26000,69.160000,70.570000,52.800000,32.950000,70.150000,80.675000,87.500000,44.110000,56.095000,73.970000,8.050000,40.420000,85.550000,86.810000,71.462500
max,2020.000000,80.860000,68.050000,64.62000,71.210000,71.610000,56.800000,35.600000,72.430000,80.720000,88.090000,45.560000,56.530000,74.060000,8.180000,42.800000,87.210000,87.380000,71.920000


In [129]:
#Considering the null values. Imputation.
df.interpolate(method='linear', limit_direction='backward', inplace=True)



In [130]:
df.interpolate(method='linear', limit_direction='forward', inplace=True)


In [131]:
df.head()

,year,IVD,HbA1c <=9% of Total Diabetics,Hypertensive Patients w BP < 140/90,Children Weight,Depression,Dental Sealant,HbA1c >9% of Total Diabetics,Adult Weight,CAD,Tobacco,Colorectal Cancer,Pap Tests,Early Entry Prenatal,Low Birth Weight,Immunization,HIV and Received Follow-up Treatment,Asthmatic,Statin Therapy
0,2016,78.40,67.90,62.39,62.92,60.34,48.71,32.10,62.46,79.46,85.19,39.89,54.36,74.06,7.80,42.80,83.17,87.38,70.09
1,2017,79.27,67.05,62.71,65.85,66.15,50.71,32.95,63.85,80.72,87.50,42.02,55.67,73.97,8.03,40.24,84.52,86.62,70.09
2,2018,80.86,67.21,63.26,69.16,70.57,52.80,32.79,70.15,80.63,88.09,44.11,55.95,73.82,8.00,39.44,85.55,86.58,70.09
3,2019,80.78,68.05,64.62,71.21,71.61,56.80,31.95,72.43,80.63,87.17,45.56,56.53,73.81,8.05,39.75,87.21,85.95,70.09
4,2020,78.80,35.60,57.98,65.13,64.21,48.68,35.60,65.72,80.63,83.43,40.09,56.53,73.48,8.18,40.42,81.41,85.95,71.92


In [132]:
df.set_index('year',inplace= True) 

In [133]:
from statsmodels.tsa.api import VAR
x = df.iloc[:-1,0:17] #slicing , -1 = 2020 exclude, -2 = 2019 exclude

In [134]:
x.head()

,IVD,HbA1c <=9% of Total Diabetics,Hypertensive Patients w BP < 140/90,Children Weight,Depression,Dental Sealant,HbA1c >9% of Total Diabetics,Adult Weight,CAD,Tobacco,Colorectal Cancer,Pap Tests,Early Entry Prenatal,Low Birth Weight,Immunization,HIV and Received Follow-up Treatment,Asthmatic
year,,,,,,,,,,,,,,,,,
2016,78.40,67.90,62.39,62.92,60.34,48.71,32.10,62.46,79.46,85.19,39.89,54.36,74.06,7.80,42.80,83.17,87.38
2017,79.27,67.05,62.71,65.85,66.15,50.71,32.95,63.85,80.72,87.50,42.02,55.67,73.97,8.03,40.24,84.52,86.62
2018,80.86,67.21,63.26,69.16,70.57,52.80,32.79,70.15,80.63,88.09,44.11,55.95,73.82,8.00,39.44,85.55,86.58
2019,80.78,68.05,64.62,71.21,71.61,56.80,31.95,72.43,80.63,87.17,45.56,56.53,73.81,8.05,39.75,87.21,85.95


In [135]:
x.describe()

,IVD,HbA1c <=9% of Total Diabetics,Hypertensive Patients w BP < 140/90,Children Weight,Depression,Dental Sealant,HbA1c >9% of Total Diabetics,Adult Weight,CAD,Tobacco,Colorectal Cancer,Pap Tests,Early Entry Prenatal,Low Birth Weight,Immunization,HIV and Received Follow-up Treatment,Asthmatic
count,4.00000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.0000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,79.82750,67.552500,63.245000,67.285000,67.167500,52.255000,32.447500,67.2225,80.360000,86.987500,42.895000,55.627500,73.915000,7.970000,40.557500,85.112500,86.632500
std,1.20026,0.496009,0.984564,3.653022,5.130396,3.459677,0.496009,4.8216,0.601498,1.257309,2.474813,0.917764,0.121244,0.115181,1.530847,1.704433,0.585228
min,78.40000,67.050000,62.390000,62.920000,60.340000,48.710000,31.950000,62.4600,79.460000,85.190000,39.890000,54.360000,73.810000,7.800000,39.440000,83.170000,85.950000
25%,79.05250,67.170000,62.630000,65.117500,64.697500,50.210000,32.062500,63.5025,80.337500,86.675000,41.487500,55.342500,73.817500,7.950000,39.672500,84.182500,86.422500
50%,80.02500,67.555000,62.985000,67.505000,68.360000,51.755000,32.445000,67.0000,80.630000,87.335000,43.065000,55.810000,73.895000,8.015000,39.995000,85.035000,86.600000
75%,80.80000,67.937500,63.600000,69.672500,70.830000,53.800000,32.830000,70.7200,80.652500,87.647500,44.472500,56.095000,73.992500,8.035000,40.880000,85.965000,86.810000
max,80.86000,68.050000,64.620000,71.210000,71.610000,56.800000,32.950000,72.4300,80.720000,88.090000,45.560000,56.530000,74.060000,8.050000,42.800000,87.210000,87.380000


In [136]:
df.shape

(5, 18)

In [137]:
x_var = VAR(x)
model = x_var.fit()

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:215: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [138]:
pred = model.forecast(model.y,steps=1) #step 1 = 1 years, step 2 = next 2 years

/usr/local/lib/python3.7/dist-packages/statsmodels/base/wrapper.py:35: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [139]:
print(pred)

[[80.8598437  68.64217944 65.41418814 72.12097841 71.95472336 58.6857786
  31.66622081 73.2055552  80.88611275 86.95676719 46.21031688 56.95428465
  74.04533725  8.10085448 40.08478725 88.17539476 85.92171334]]


In [140]:
#merge and graph
#score

def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [141]:
MAPE(68.05, 67.81) #Error, 98%accuracy

0.3526818515797133

In [153]:
res.to_csv('predicted.csv')
df.to_csv('actual.csv')